In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "macro_f1_score":   "$MACRO_F1",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision", "macro_f1_score" ] + params
        return df[cols]
    return df

In [5]:
def get_window_classifier_results(prefix):
    collections = "WINDOW_CLASSIFIER_BR,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS".split(",")
    dfs = []
    for c in collections:
        col = prefix + c
        print(col)
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + prefix[:-1]] = col.replace(prefix,"")
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

In [6]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Generate Per Class Metrics

In [7]:
def get_algo_name(coll):
    if "WINDOW" in coll:
        return "Window-Based Tagger"
    if "CRF" in coll:
        return "CRF"
    if "HMM" in coll:
        return "HMM"
    if "PERCEPTRON" in coll:
        return "Structured Perceptron"
    if "RNN" in coll:
        return "Bidirectional RNN"

In [8]:
from collections import namedtuple

Metric = namedtuple("Metric", "code f1 prec rec")
Metric(code="50", f1=0.99, prec=0.98, rec=0.97)

Metric(code='50', f1=0.99, prec=0.98, rec=0.97)

In [18]:
def sort_key(metric):
    code = metric.code
    if "b" in code:
        code = code.replace("b", ".1")
    return float(code)

def compute_metrics_by_class(coll):
    print(coll)
    print("code\t\tf1\trec\tprec")
    for row in db[coll].find({}):
        keys = []        

        metrics = []
        for k in row.keys():
            if k[0].isdigit():
                keys.append(k)
                code, f1, prec, rec = k, row[k]["f1_score"], row[k]["precision"], row[k]["recall"]
                metric = Metric(code=code, f1=f1, prec=prec, rec=rec)
                metrics.append(metric)
        
        for metric in sorted(metrics, key = sort_key):
            print("{code}\t{f1:.4f}\t{rec:.4f}\t{prec:.4f}".format(code=metric.code.ljust(10), f1=metric.f1, prec=metric.prec, rec=metric.rec))

        print("\n" + ",".join(sorted(keys)))
        print("")

In [21]:
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG_FIXED,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS_FIXED,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):

    #collection = collection.replace("CB","SC")
    print(get_algo_name(collection))
    compute_metrics_by_class(collection)
    print("")

Window-Based Tagger
TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS
code		f1	rec	prec
1         	0.8260	0.7960	0.8584
2         	0.7399	0.6871	0.8016
3         	0.8274	0.7814	0.8792
4         	0.8318	0.8193	0.8447
5         	0.4490	0.3113	0.8049
5b        	0.0303	0.0227	0.0455
6         	0.8357	0.8018	0.8725
7         	0.8381	0.7598	0.9344
11        	0.8995	0.8775	0.9227
12        	0.8629	0.7798	0.9659
13        	0.7340	0.6921	0.7814
14        	0.7484	0.7437	0.7532
50        	0.9041	0.8803	0.9293

1,11,12,13,14,2,3,4,5,50,5b,6,7


CRF
TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG_FIXED
code		f1	rec	prec
1         	0.8365	0.8007	0.8756
2         	0.7622	0.7415	0.7842
3         	0.8197	0.7737	0.8715
4         	0.7934	0.7922	0.7946
5         	0.3188	0.2075	0.6875
5b        	0.0000	0.0000	0.0000
6         	0.8337	0.7793	0.8964
7         	0.8263	0.7394	0.9364
11        	0.9268	0.9314	0.9223
12        	0.8629	0.7798	0.9659
13        	0.7291	0.6794	0.7868
14        	0.7002	0.6835	0.7